In [ ]:
# #To be used on cloud platforms

# ! python -c "import pykeen" || pip install pykeen
# ! python -c "import scprep" || pip install scprep

In [1]:
import os, pandas as pd, numpy as np, torch, pykeen, scprep


/usr/local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(os.getcwd())
data_dir = f"{os.getcwd()}/data"
kge_dir = f"{os.getcwd()}/KGEs"

/Users/ferran/PhD/yaleCollab


# DataBases

The integrated database is the object ingested by pykeen's *TriplesFactory*.

In [4]:
pd.read_csv(f"{data_dir}/dCellChat.tsv", sep="\t")

,head,relation,tail,pathway
0,Tgfb1,LRinteraction,Tgfbr1,TGFb
1,Tgfb1,LRinteraction,Tgfbr2,TGFb
2,Tgfb2,LRinteraction,Tgfbr1,TGFb
3,Tgfb2,LRinteraction,Tgfbr2,TGFb
4,Tgfb3,LRinteraction,Tgfbr1,TGFb
...,...,...,...,...
2058,Vcam1,LRinteraction,Itga4,VCAM
2059,Vcam1,LRinteraction,Itgb1,VCAM
2060,Vcam1,LRinteraction,Itga9,VCAM
2061,Vcam1,LRinteraction,Itgb7,VCAM


In [5]:
pd.read_csv(f"{data_dir}/dSTRING.tsv", sep="\t")

,head,relation,tail
0,Gnai3,FIstring,Prkca
1,Gnai3,FIstring,Rgs18
2,Gnai3,FIstring,Gpsm2
3,Gnai3,FIstring,Adrb3
4,Gnai3,FIstring,Gnat2
...,...,...,...
437237,Gm28635,FIstring,Arhgap35
437238,Gm28635,FIstring,Cdh17
437239,Gm28635,FIstring,Kif5b
437240,Gm28635,FIstring,Tjp3


In [7]:
pd.read_csv(f"{data_dir}/dINT_simpl.tsv", sep="\t", header=None)

,0,1,2
0,Tgfb1,LRinteraction,Tgfbr1
1,Tgfb1,LRinteraction,Tgfbr2
2,Tgfb2,LRinteraction,Tgfbr1
3,Tgfb2,LRinteraction,Tgfbr2
4,Tgfb3,LRinteraction,Tgfbr1
...,...,...,...
439018,Gm28635,FIstring,Arhgap35
439019,Gm28635,FIstring,Cdh17
439020,Gm28635,FIstring,Kif5b
439021,Gm28635,FIstring,Tjp3


# scRNAseq data

## Prep data

## Build gene-gen graph

# KGE